In [1]:
import requests
from bs4 import BeautifulSoup 
import csv
import re
import string

In [2]:
#Url Details
base_url = "https://www.imdb.com"
first_url = "https://www.imdb.com/search/title/?title_type=feature&genres=biography&explore=genres&view=advanced"

In [3]:
# File Details and the Header Details
header = ["Movie_Name","Year","Genre","Director","RunTime","Rating","Metascore","Review_url","NumofVotes"]
myfile = open("MovieReview.csv", 'w',newline="",encoding='utf-8')
csvwriter = csv.writer(myfile, delimiter =',')
csvwriter.writerow(header)
myfile.close()

In [8]:
# Main Logic to Extract All the Biographic Movies
First_run = True
next_run = False
Next_Url = ""

if First_run:
    page_content = ""
    page_response =""
    page_response,page_content=get_webpage_content(first_url)
    res_code=int(re.findall(r"[0-9]+", str(page_response))[0])

    if (res_code>=200 | res_code<300):
        get_movie_details(page_content)
        Next_Url=check_next_page(page_content)
        print(Next_Url)

print(Next_Url)
while (Next_Url):
    page_content = ""
    page_response,page_content = get_webpage_content(base_url+Next_Url)
    res_code=int(re.findall(r"[0-9]+", str(page_response))[0])
    if (res_code>=200 | res_code<300):
        get_movie_details(page_content)
        Next_Url=check_next_page(page_content)
        print(base_url+Next_Url)


/search/title/?title_type=feature&genres=biography&start=51&explore=genres
/search/title/?title_type=feature&genres=biography&start=51&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=101&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=151&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=201&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=251&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=301&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=351&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=401&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=451&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=501&e

https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4301&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4351&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4401&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4451&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4501&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4551&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4601&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4651&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4701&explore=genres
https://www.imdb.com/search/title/?title_type=feature&genres=biography&start=4751&explore=genres
https://www.imdb.com/search/ti

In [5]:
# Get the page content in a HTML Format
def get_webpage_content(url):
    response = requests.get(url)
    page_content = BeautifulSoup(response.text,'html.parser')
    return (response, page_content)

In [6]:
# Get the Next Page URL
def check_next_page(web_content):
    next_page_url = ""
    if web_content.find('a',{'class':'lister-page-next next-page'}):
        next_page_url = web_content.find('a',{'class':'lister-page-next next-page'})['href']
        return next_page_url
    else:
        next_page_url =""
        return next_page_url

In [7]:
# Get All the Movie Details
def get_movie_details(web_content):
    Movie_Det = web_content.findAll('div',{'lister-item-content'})

    for i in Movie_Det:
        Name= i.find('a').text.replace(',',"")
        movie_link = i.find('a')['href']
        Review_url='https://imdb.com' + movie_link + 'reviews'

        if i.find('span',{'lister-item-year'}):
            year_det = i.find('span',{'lister-item-year'}).text

            if (year_det):
                if (re.search(r"[\d]+",str(year_det))):
                    Year = re.search(r"[\d]+",str(year_det))[0]
                elif (re.findall(r"[a-zA-Z]+",str(year_det))):
                    Year = re.findall(r"[a-zA-Z]+",str(year_det))[0]
                else:
                     Year = "NA"
            else:
                Year = "NA"
        else:
            Year ="NA"
        if i.find('span',{'runtime'}):
            RunTime= i.find('span',{'runtime'}).text
        else:
            RunTime = "NA"
        if i.find('div',{'inline-block ratings-imdb-rating'}):
            Rating= i.find('div',{'inline-block ratings-imdb-rating'})['data-value']
        else:
            Rating = "NA"
        if i.find('div',{'inline-block ratings-metascore'}):
            Metascore= i.find('div',{'inline-block ratings-metascore'}).span.text.strip()
        else:
            Metascore= '0'
        if i.find('span',{'name':'nv'}):
            NumofVotes=i.find('span',{'name':'nv'})['data-value']
        else:
            NumofVotes = "NA"
        if (i.find('p',class_='').find_all('a')):
            Director = i.find('p',class_='').find_all('a')[0].text
        else:
            Director = "NA"
        if(i.find('span',{'genre'})):
            Genre_det = i.find('span',{'genre'}).text
            Genre = Genre_det.translate(str.maketrans('','',string.punctuation)).strip()
            Genre = re.sub(r"(?:Drama)","",Genre)
        else:
            Genre = "NA"
        
        if (Year != 'NA' or RunTime != "NA"):
            myfile =open("MovieReview.csv", 'a',encoding='utf-8', newline="")
            csvwriter = csv.writer(myfile)
            myfile.write(Name+','+Year+','+Genre+','+Director+','+RunTime+','+Rating+','+Metascore+','+Review_url+','+NumofVotes+'\n')
            myfile.close()

In [69]:
with open("MovieReview.csv",encoding='utf-8') as iFile:
    print(iFile.read())

Movie_Name,Year,Genre,Director,RunTime,Rating,Metascore,Review_url,NumofVotes
Judas and the Black Messiah,2021,Biography  History,Shaka King,126 min,7.7,85,https://imdb.com/title/tt9784798/reviews,23517
The Dig,2021,Biography  History,Simon Stone,112 min,7.1,73,https://imdb.com/title/tt3661210/reviews,45040
The United States vs. Billie Holiday,2021,Biography  Music,Lee Daniels,130 min,6.4,52,https://imdb.com/title/tt8521718/reviews,3559
The Wolf of Wall Street,2013,Biography Crime ,Martin Scorsese,180 min,8.2,75,https://imdb.com/title/tt0993846/reviews,1203392
Mank,2020,Biography Comedy ,David Fincher,131 min,7,79,https://imdb.com/title/tt10618286/reviews,42813
Trial by Fire,2018,Biography ,Edward Zwick,127 min,7,51,https://imdb.com/title/tt3263946/reviews,4616
The Greatest Showman,2017,Biography  Musical,Michael Gracey,105 min,7.6,48,https://imdb.com/title/tt1485796/reviews,242603
Hamilton,2020,Biography  History,Thomas Kail,160 min,8.5,90,https://imdb.com/title/tt8503618/reviews,5963